In [8]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import datetime

from GCNN-DDGF.utils import normalize_adj, StandardScaler

SyntaxError: invalid syntax (<ipython-input-8-8c9b3d5be003>, line 13)

# Model Architecture

In [3]:
# Create model
def gcn(x, weights, biases):
    
    x = tf.transpose(x, [1, 0, 2]) # 272, ?, 24
    x = tf.reshape(x, [sn, -1]) # 207, batch*feature_num
    Atem1 = 0.5*(weights['A1'] + tf.transpose(weights['A1']))#+ Atem 
    Atem1 = normalize_adj(Atem1)

    Z1 = tf.matmul(Atem1, x) # 272, batch*feature_num 
    Z1 = tf.reshape(Z1, [-1, feature_in]) # 207* 100, frequency
    #layer_1 = tf.matmul(Z1, weights['h1']) 
    layer_1 = tf.add(tf.matmul(Z1, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1) # 207*100, hidden1

    
    Atem2 = 0.5*(weights['A2'] + tf.transpose(weights['A2']))#+ Atem 
    Atem2 = normalize_adj(Atem2)
    
    layer_1 = tf.reshape(layer_1, [sn, -1])  # 207, batchsize*hidden1
    Z2 = tf.matmul(Atem2, layer_1)
    Z2 = tf.reshape(Z2, [-1, n_hidden_vec1]) # 207*batchsize, n_hidden_vec1
    layer_2 = tf.add(tf.matmul(Z2, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2) # 207*batchsize, hidden2

    Atem3 = 0.5*(weights['A3'] + tf.transpose(weights['A3']))#+ Atem 
    Atem3 = normalize_adj(Atem3) 
    
    layer_2 = tf.reshape(layer_2, [sn, -1])  # 207, batchsize*hidden2
    Z3 = tf.matmul(Atem3, layer_2)
    Z3 = tf.reshape(Z3, [-1, n_hidden_vec2]) # 207*batchsize, hidden2
    layer_3 = tf.add(tf.matmul(Z3, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3) # 207*batchsize, hidden3
    
    out_layer = tf.add(tf.matmul(layer_3, weights['out']), biases['bout'])  # 207*100, horizon
    out_layer = tf.reshape(out_layer, [sn, -1, horizon]) # 207, 100, horizon
    out_layer = tf.transpose(out_layer, [1, 0, 2]) # 100, 207, horizon
    out_layer = tf.reshape(out_layer, [-1, sn*horizon]) # 100, 207*horizon
 
    return out_layer

In [4]:
def gcn_corr_final(feature_in, horizon, learning_rate, decay, batch_size, n_hidden_vec1, n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs):
   
    feature_in = int(feature_in) 
    
    early_stop_k = 0 # early stop patience
    display_step = 1 # frequency of printing results
    best_val = 10000
    traing_error = 0
    test_error = 0
    predic_res = []

    tf.reset_default_graph()

    batch_size = int(batch_size)
    n_hidden_vec1 = int(n_hidden_vec1)
    n_hidden_vec2 = int(n_hidden_vec2)
    n_hidden_vec3 = int(n_hidden_vec3)
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, sn, feature_in]) # X is the input signal
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output
    
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([feature_in, n_hidden_vec1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_vec1, n_hidden_vec2])),
        'h3': tf.Variable(tf.random_normal([n_hidden_vec2, n_hidden_vec3])),
        'out': tf.Variable(tf.random_normal([n_hidden_vec3, 1])),
        'A1': tf.Variable(tf.random_normal([sn, sn])),
        'A2': tf.Variable(tf.random_normal([sn, sn])),
        'A3': tf.Variable(tf.random_normal([sn, sn])),
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([1, n_hidden_vec1])),
        'b2': tf.Variable(tf.random_normal([1, n_hidden_vec2])), 
        'b3': tf.Variable(tf.random_normal([1, n_hidden_vec3])),
        'bout': tf.Variable(tf.random_normal([1, horizon])), 
    }

    # Construct model
    pred= gcn(X, weights, biases)
    pred = scaler.inverse_transform(pred)
    Y_true_tr = scaler.inverse_transform(Y)
    cost = tf.reduce_mean(tf.pow(pred - Y_true_tr, 2)) 

    pred_val= gcn(X, weights, biases)
    pred_val = scaler.inverse_transform(pred_val)
    Y_true_val = scaler.inverse_transform(Y)
    cost_val =  tf.reduce_mean(tf.pow(pred_val - Y_true_val, 2)) 

    pred_tes= gcn(X, weights, biases)
    pred_tes = scaler.inverse_transform(pred_tes)
    Y_true_tes = scaler.inverse_transform(Y)
    cost_tes = tf.reduce_mean(tf.pow(pred_tes - Y_true_tes, 2)) 
                                         
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(num_train/batch_size)

            for i in range(total_batch):
                
                _, c = sess.run([optimizer, cost], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,], 
                                                              keep_prob: keep})

                avg_cost += c * batch_size #/ total_batch 
                
            # rest part of training dataset
            if total_batch * batch_size != num_train:
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[total_batch*batch_size:num_train,], 
                                          Y: Y_training[total_batch*batch_size:num_train,],
                                                  keep_prob: keep})
                avg_cost += c * (num_train - total_batch*batch_size)
            
            avg_cost = np.sqrt(avg_cost / num_train)
            #Display logs per epoch step
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training RMSE=", \
                    "{:.9f}".format(avg_cost))
            # validation
            c_val = sess.run([cost_val], feed_dict={X: X_val, Y: Y_val,  keep_prob:1})
            c_val = np.sqrt(c_val[0])
            print("Validation RMSE: ", c_val)
            # testing
            c_tes, preds, A1, Y_true = sess.run([cost_tes, pred_tes, weights['A1'], Y_true_tes], feed_dict={X: X_test,Y: Y_test, keep_prob: 1})
            c_tes = np.sqrt(c_tes)

            if c_val < best_val:
                best_val = c_val
                # save model
                #saver.save(sess, './bikesharing_gcnn_ddgf')
                test_error = c_tes
                traing_error = avg_cost
                predic_res = preds
                early_stop_k = 0 # reset to 0

            # update early stopping patience
            if c_val >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
                break
            

        print("epoch is ", epoch)
        print("training RMSE is ", traing_error)
        print("Optimization Finished! the lowest validation RMSE is ", best_val)
        print("The test RMSE is ", test_error)
    
    #test_Y = Y_test
    #test_error = np.sqrt(test_error)
    return best_val, predic_res,Y_true,test_error, A1

# Import Data

In [5]:
file_Name = "data/NYCBikeHourly272.pickle"
fileObject = open(file_Name,'rb') 
hourly_bike = pickle.load(fileObject) 
hourly_bike = pd.DataFrame(hourly_bike)

# Split Data into Training, Validation and Testing

In [6]:
sn = 272 # station number 
feature_in = 24 # number of features at each node, e.g., bike sharing demand from past 24 hours
horizon = 1 # the length to predict, e.g., predict the future one hour bike sharing demand

X_whole = []
Y_whole = []

x_offsets = np.sort(
    np.concatenate((np.arange(-feature_in+1, 1, 1),))
)

y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

min_t = abs(min(x_offsets))
max_t = abs(hourly_bike.shape[0] - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = hourly_bike.iloc[t + x_offsets, 0:sn].values.flatten('F')
    y_t = hourly_bike.iloc[t + y_offsets, 0:sn].values.flatten('F')
    X_whole.append(x_t)
    Y_whole.append(y_t)

X_whole = np.stack(X_whole, axis=0)
Y_whole = np.stack(Y_whole, axis=0)

n_input_vec = X_whole.shape[1] # e.g., 272 * 24
n_output_vec = Y_whole.shape[1] # each row represent a result


X_whole = np.reshape(X_whole, [X_whole.shape[0], sn, feature_in])
num_samples = X_whole.shape[0]
num_train = 20000 # Note here actually we use the first 20000 to train the model. The paper mentioned "22304" need to be corrected.
num_val = 2000
num_test = 2000

X_training = X_whole[:num_train, :]
Y_training = Y_whole[:num_train, :]

# shuffle the training dataset
perm = np.arange(X_training.shape[0])
np.random.shuffle(perm)
X_training = X_training[perm]
Y_training = Y_training[perm]

X_val = X_whole[num_train:num_train+num_val, :]
Y_val = Y_whole[num_train:num_train+num_val, :]

X_test = X_whole[num_train+num_val:num_train+num_val+num_test, :]
Y_test = Y_whole[num_train+num_val:num_train+num_val+num_test, :]



In [7]:
scaler = StandardScaler(mean=X_training.mean(), std=X_training.std())

X_training = scaler.transform(X_training)
Y_training = scaler.transform(Y_training)

X_val = scaler.transform(X_val)
Y_val = scaler.transform(Y_val)

X_test = scaler.transform(X_test)
Y_test = scaler.transform(Y_test)

# Hyperparameters

In [20]:
sn = 272 # station number 
feature_in = 24 # number of features at each node, e.g., bike sharing demand from past 24 hours
horizon = 1 # the length to predict, e.g., predict the future one hour bike sharing demand

learning_rate = 0.005 # learning rate
decay = 0.9
batchsize = 100 # batch size 

n_hidden_vec1 = 10 # feature length of the first hidden layer after the graph convolution
n_hidden_vec2 = 10 # feature length of the second ...
n_hidden_vec3 = 5 # feature length of the third ...

keep = 1 # drop out probability

early_stop_th = 20 # early stopping threshold, if validation RMSE not dropping in continuous 20 steps, break
training_epochs = 200 # total training epochs


# Training

In [21]:
a = datetime.datetime.now()

val_error, predic_res, test_Y, test_error, A1 = gcn_corr_final(feature_in, horizon, learning_rate, decay, batchsize, n_hidden_vec1,
                                                            n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs)


b = datetime.datetime.now()

print('Total training time: ', b-a)

#np.savetxt("prediction.csv", predic_res, delimiter = ',')
#np.savetxt("prediction_Y.csv", test_Y, delimiter = ',')


Epoch: 0001 Training RMSE= 4291.778081102
Validation RMSE:  5.006561
Epoch: 0002 Training RMSE= 17.114909205
Validation RMSE:  3.9244106
Epoch: 0003 Training RMSE= 13.553793941
Validation RMSE:  3.7608411
Epoch: 0004 Training RMSE= 12.750441022
Validation RMSE:  3.6765308
Epoch: 0005 Training RMSE= 12.160855937
Validation RMSE:  3.5996826
Epoch: 0006 Training RMSE= 11.529853919
Validation RMSE:  3.5566764
Epoch: 0007 Training RMSE= 11.026012301
Validation RMSE:  3.5474122
Epoch: 0008 Training RMSE= 10.702007723
Validation RMSE:  3.5314665
Epoch: 0009 Training RMSE= 10.472437596
Validation RMSE:  3.506932
Epoch: 0010 Training RMSE= 10.273559878
Validation RMSE:  3.4797187
Epoch: 0011 Training RMSE= 10.085142627
Validation RMSE:  3.4496176
Epoch: 0012 Training RMSE= 9.911544406
Validation RMSE:  3.4167523
Epoch: 0013 Training RMSE= 9.759352119
Validation RMSE:  3.3867931
Epoch: 0014 Training RMSE= 9.627315130
Validation RMSE:  3.3617408
Epoch: 0015 Training RMSE= 9.519801097
Validation R

Epoch: 0124 Training RMSE= 7.616563468
Validation RMSE:  3.0511768
Epoch: 0125 Training RMSE= 7.613029199
Validation RMSE:  3.0499465
Epoch: 0126 Training RMSE= 7.609035864
Validation RMSE:  3.05067
Epoch: 0127 Training RMSE= 7.605024359
Validation RMSE:  3.047388
Epoch: 0128 Training RMSE= 7.601820745
Validation RMSE:  3.0490186
Epoch: 0129 Training RMSE= 7.597770553
Validation RMSE:  3.0475662
Epoch: 0130 Training RMSE= 7.594134526
Validation RMSE:  3.0471315
Epoch: 0131 Training RMSE= 7.590749385
Validation RMSE:  3.0440488
Epoch: 0132 Training RMSE= 7.587226932
Validation RMSE:  3.0443652
Epoch: 0133 Training RMSE= 7.583420856
Validation RMSE:  3.0431318
Epoch: 0134 Training RMSE= 7.580357585
Validation RMSE:  3.0409727
Epoch: 0135 Training RMSE= 7.576216023
Validation RMSE:  3.0402184
Epoch: 0136 Training RMSE= 7.573519866
Validation RMSE:  3.0397015
Epoch: 0137 Training RMSE= 7.569714508
Validation RMSE:  3.0393634
Epoch: 0138 Training RMSE= 7.566621339
Validation RMSE:  3.037732

In [ ]:
# use three layers: 2.34